# Hexapod movies
Craig Lage - 03-Jun-24 \


In [ ]:
import sys, time, os, asyncio, glob
import shlex, subprocess
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient


## Set up the necessary subroutines

In [ ]:
def plotHexapodContour(df, ax, index, hex="Camera"):
    ax.set_xlabel("X (um)")
    ax.set_ylabel("Y (um)")
    ax.set_title(f"{hex} Hexapod motions", fontsize=18)
    xmin = -5.0; xmax = 5.0; ymin = -5.0; ymax = 5.0
    zmin = -80.0; zmax = 20.0
    levels = np.linspace(zmin, zmax, 51)
    cbarticks = np.linspace(zmin, zmax, 11)
    cbarlabels = []
    for tick in cbarticks:
        cbarlabels.append(f"{tick:.1f}")
    theta = np.linspace(0, 2 * np.pi, 51)
    x = xmax*np.cos(theta)
    y = ymax*np.sin(theta)
    if hex == "Camera":
        rmin = 0.0
    elif hex == "M2":
        rmin = 0.5
        
    r = np.linspace(rmin, xmax, 51)
    R, Theta = np.meshgrid(r, theta)
    Xcen = df.iloc[index]["position0"]
    Ycen = df.iloc[index]["position1"]
    Zcen = df.iloc[index]["position2"]
    Z = Zcen * np.ones_like(R)
    
    # Express the mesh in the cartesian system.
    X, Y = R*np.cos(Theta), R*np.sin(Theta)
    X += Xcen
    Y += Ycen
    Phi = 0.0 # Placeholder for u, v
    X = X * np.cos(Phi) - Z * np.sin(Phi)
    Z = -X * np.sin(Phi) + Z * np.cos(Phi)
    # Plot the surface.
    im = ax.contourf(X, Y, Z, levels=levels)
    ax.scatter(Xcen,Ycen, marker='+', s=200, color='black')

    ax.plot(x,y, color='black')
    ax.scatter(0.0,0.0, marker='+', s=200, color='black')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.set_ticks(cbarticks)
    cbar.set_ticklabels(cbarlabels)
    ax.set_xlim(1.5*xmin, 1.5*xmax)
    ax.set_ylim(1.5*ymin, 1.5*ymax)


## Define the times and options

In [ ]:
# Times to start looking at values
start = Time("2024-05-07T01:17:54Z", scale='utc')
end = Time("2024-05-07T01:26:08Z", scale='utc')

frameN = 10

## Now generate the frames
### This will take some time

In [ ]:
client = EfdClient('usdf_efd')

timestamp = start.isot.split('.')[0].replace('-','').replace(':','')
dirName = f"/home/c/cslage/u/Hexapods/movies/movie_{timestamp}"
%mkdir -p {dirName}
movieName = f"hexapod_movie_{timestamp}.mp4"

camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
print(len(camhex))
m2hex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=2)
print(len(m2hex))

fig = plt.figure(figsize=(8,8))
# Now build the individual frames
for n in range(0, len(camhex), frameN):
    ax = fig.add_subplot(1,1,1)
    plotHexapodContour(camhex, ax, n, hex="Camera")
    plt.savefig(f"{dirName}/Frame_{(n + 1):05d}.png")
    plt.clf()
    nFrames = int(n / frameN)
    if nFrames%100 == 0:
        print(f"{nFrames} frames done")


## Now build the movie

In [ ]:
movie_name = f"hexapod_movie_{timestamp}.mp4"
print(f"\033[1mThe movie name will be: {dirName}/{movie_name}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movie_name}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()